In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [4]:
train_path="C:\\Users\\tusha\\Downloads\\AID710\\BC2\\RSNA_Crop\\BC2"
test_path="C:\\Users\\tusha\\Downloads\\AID710\\BC2\\RSNA_Crop\\test"

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [5]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['cancer', 'no_cancer']

In [6]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        

In [7]:
model=ConvNet(num_classes=2).to(device)

In [8]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [9]:
num_epochs=25

In [10]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.png'))

In [11]:
print(train_count,test_count)

800 240


In [ ]:
import matplotlib.pyplot as plt

# Initialize lists for plotting
train_accuracy_list = []
train_loss_list = []
test_accuracy_list = []

best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)

        train_accuracy += int(torch.sum(prediction == labels.data))

    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count

    # Save values for plotting
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)

    model.eval()
    test_accuracy = 0.0

    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))

    test_accuracy = test_accuracy / test_count
    
    # Save test accuracy for plotting
    test_accuracy_list.append(test_accuracy)

    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) + ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))

    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy

# Plotting
plt.figure(figsize=(10, 5))

# Plotting Train and Test accuracy
plt.subplot(1, 2, 1)
plt.plot(range(num_epochs), train_accuracy_list, label='Train Accuracy')
plt.plot(range(num_epochs), test_accuracy_list, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Train loss
plt.subplot(1, 2, 2)
plt.plot(range(num_epochs), train_loss_list, label='Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


Epoch: 0 Train Loss: tensor(19.1554) Train Accuracy: 0.47625 Test Accuracy: 0.5041666666666667
Epoch: 1 Train Loss: tensor(7.1144) Train Accuracy: 0.545 Test Accuracy: 0.5333333333333333
Epoch: 2 Train Loss: tensor(4.3014) Train Accuracy: 0.61375 Test Accuracy: 0.4875
Epoch: 3 Train Loss: tensor(2.1046) Train Accuracy: 0.69125 Test Accuracy: 0.4708333333333333
Epoch: 4 Train Loss: tensor(0.8621) Train Accuracy: 0.77375 Test Accuracy: 0.4791666666666667
Epoch: 5 Train Loss: tensor(1.5555) Train Accuracy: 0.73625 Test Accuracy: 0.5083333333333333
Epoch: 6 Train Loss: tensor(0.8572) Train Accuracy: 0.805 Test Accuracy: 0.4708333333333333
Epoch: 7 Train Loss: tensor(0.5187) Train Accuracy: 0.87625 Test Accuracy: 0.49166666666666664
Epoch: 8 Train Loss: tensor(0.3032) Train Accuracy: 0.9075 Test Accuracy: 0.475
Epoch: 9 Train Loss: tensor(0.2071) Train Accuracy: 0.935 Test Accuracy: 0.4708333333333333
Epoch: 10 Train Loss: tensor(0.1555) Train Accuracy: 0.95125 Test Accuracy: 0.466666666666